# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import pickle

# Kriging 3D Interpolation

### Define Functions

In [ ]:
def plot_3D_mesh(t, grid_x, grid_y, grid_z, title: str, color_scale='Viridis'):
    GZ, GY, GX = np.meshgrid(grid_z, grid_y, grid_x, indexing='ij')

    fig = go.Figure(
        data=[go.Volume(
            x=GX.flatten(),
            y=GY.flatten(),
            z=GZ.flatten(),
            value=t.flatten(),
            isomax=t.max(),
            isomin=t.min(),
            opacity=0.5,  
            surface_count=25,  # Adjust the number of isosurfaces
            colorscale=color_scale 
        )])

    # Update the layout of the plot
    fig.update_layout(
        height=600,
        width=700,
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis=dict(
                title='Depth',
                autorange='reversed'  # Automatically reverse the z-axis
            )
        )
    )

    # Show the plot
    fig.show()

In [ ]:
def load_data(name: str) -> pd.DataFrame:
    with open(f'data/t_{name}.pkl', 'rb') as f:
        interpolated_data = pickle.load(f)
    with open(f'data/ss3d_{name}.pkl', 'rb') as f:
        varience_data = pickle.load(f)
    return interpolated_data, varience_data

In [ ]:
def load_grid() -> np.ndarray:
    with open(f'data/grid.pkl', 'rb') as f:
        grid = pickle.load(f)
    return grid

### Load Data

In [ ]:
grid = load_grid()
t_poro, ss3d_poro = load_data('poro')
t_hydr, ss3d_hydr = load_data('hydr')

### Kriging Interpolation of Porosity

In [ ]:
plot_3D_mesh(t_poro, *grid, "Kriging Interpolation of Porosity", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_poro, *grid, "Kriging Varience of Porosity")

### Kriging Interpolation of Hydrate Saturation

In [ ]:
plot_3D_mesh(t_hydr, *grid, "Kriging Interpolation of Hydrate Saturation", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_hydr, *grid, "Kriging Varience of Hydrate Saturation")